# Sentiment Analysis with XGBoost on Algorithmia

With this notebook, we will be training an XGBoost model on Amazon's Musical Instrument Reviews dataset and be able to use this model to predict the sentiment of the given texts. If you would like to see the final product first, you can check out this algorithm in action at https://algorithmia.com/algorithms/asli/xgboost_basic_sentiment_analysis

## Overview

In this notebook, we will step by step: 

1. Load the training data

2. Preprocess the data

3. Setup an XGBoost model and do a mini hyperparameter search

4. Fit the data on our model

5. Get the predictions

6. Check the accuracy

7. Pickle the final model

8. Create the inference script and the dependency file of our algorithm on Algorithmia

To get our model file and the inference scripts uploaded to Algorithmia and get linked together, we will be persisting them on paths that we'll read from our Github Actions workflow YAML file. Our utility functions will be able to retrieve that information when this notebook is executed on Github's workflow machines, so no need to worry about internals!

## Building the XGBoost model

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline

from string import punctuation
from nltk.corpus import stopwords

from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import joblib

### Load the training data
Let's load our training data, take a look at a few rows and one of the review texts in detail.

In [ ]:
data = pd.read_csv("./data/amazon_musical_reviews/Musical_instruments_reviews.csv")
data.head()

In [ ]:
data["reviewText"].iloc[1]

### Preprocessing
Time to process our texts! Basically, we'll:
- Remove the English stopwords
- Remove punctuations
- Drop unused columns

In [ ]:
import nltk
nltk.download('stopwords')

def threshold_ratings(data):
    def threshold_overall_rating(rating):
        return 0 if int(rating)<=3 else 1
    data["overall"] = data["overall"].apply(threshold_overall_rating)

def remove_stopwords_punctuation(data):
    data["review"] = data["reviewText"] + data["summary"]

    puncs = list(punctuation)
    stops = stopwords.words("english")

    def remove_stopwords_in_str(input_str):
        filtered = [char for char in str(input_str).split() if char not in stops]
        return ' '.join(filtered)

    def remove_punc_in_str(input_str):
        filtered = [char for char in input_str if char not in puncs]
        return ''.join(filtered)

    def remove_stopwords_in_series(input_series):
        text_clean = []
        for i in range(len(input_series)):
            text_clean.append(remove_stopwords_in_str(input_series[i]))
        return text_clean

    def remove_punc_in_series(input_series):
        text_clean = []
        for i in range(len(input_series)):
            text_clean.append(remove_punc_in_str(input_series[i]))
        return text_clean

    data["review"] = remove_stopwords_in_series(data["review"].str.lower())
    data["review"] = remove_punc_in_series(data["review"].str.lower())

def drop_unused_colums(data):
    data.drop(['reviewerID', 'asin', 'reviewerName', 'helpful', 'unixReviewTime', 'reviewTime', "reviewText", "summary"], axis=1, inplace=True)

def preprocess_reviews(data):
    remove_stopwords_punctuation(data)
    threshold_ratings(data)
    drop_unused_colums(data)

In [ ]:
preprocess_reviews(data)
data.head()

### Split our training and test sets

In [ ]:
rand_seed = 42
X = data["review"]
y = data["overall"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rand_seed)

### Mini randomized search
Let's set up a very basic cross-validated randomized search over parameter settings.

In [ ]:
params = {"max_depth": range(9,12), "min_child_weight": range(5,8)}
rand_search_cv = RandomizedSearchCV(XGBClassifier(), param_distributions=params, n_iter=1)

### Pipeline to vectorize, transform and fit
Time to vectorize our data, transform it and then fit our model to it.
To be able to feed the text data as numeric values to our model, we will first convert our texts into a matrix of token counts using a CountVectorizer. Then we will convert the count matrix to a normalized tf-idf (term-frequency times inverse document-frequency) representation. Using this transformer, we will be scaling down the impact of tokens that occur very frequently, because they convey less information to us. On the contrary, we will be scaling up the impact of the tokens that occur in a small fraction of the training data because they are more informative to us.

In [ ]:
model  = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('model', rand_search_cv)
])
model.fit(X_train, y_train)

### Predict and calculate accuracy

In [ ]:
predictions = model.predict(X_test)
acc = accuracy_score(y_test, predictions)
print(f"Model Accuracy: {round(acc * 100, 2)}")

### Automated Deployment via Github Actions
TODO: Update the instructions further
- Import the Algorithmia Github Action config utility script
- Get the model file path to be saved from the YAML file - no need to manually enter it here.
- Get the algorithm name from the YAML file too.
- If desired, overwrite the algorithm script and the dependency file from within your notebook too.

We should save the created model object to a local path. The Github action will then take the model object from this path and upload it to Algorithmia.

In [ ]:
joblib.dump(model, "model.pkl", compress=True)

## Testing inference code (the algorithm)

We will convert our inference (Algorithm) notebook named `xgboost_automated.ipynb` into a Python script named `xgboost_automated.py` - with the same name as our algorithm. 

And this is exactly what the Github Action would do if it doesn't find a `xgboost_automated.py` file in our repository. In that case, it would make this conversion for us and then push this file to the algorithm `src` directory at Algorithmia.

After the conversion, we will test that it works by simply executing this file with the `python3` executable. The script will begin its execution through our `if __name__ == "__main__"` line. 

In [1]:
# !jupyter nbconvert --to script xgboost_automated.ipynb
# !python3 xgboost_automated.py

[NbConvertApp] Converting notebook demo_algo.ipynb to script
[NbConvertApp] Writing 3362 bytes to demo_algo.py
Model file MD5 assertion done.
Model object pipeline steps assertion done
All assertions are okay, we have a perfectly uploaded model!
'Calling algorithm apply() func with input: This guitar is great!'
{'predicting_model_metadata': {'model_file': 'asli/xgboost_automated/autodeployed_model_348410b2f7071377de060f030cca7d984b0f2dca.pkl',
                               'origin_commit_SHA': '348410b2f7071377de060f030cca7d984b0f2dca',
                               'origin_commit_msg': 'Reverted multiple '
                                                    'workflow testing',
                               'origin_repo': 'aslisabanci/demo_autodeploy_algo_on_algorithmia'},
 'sentiment': 1}


## Final checks before committing this notebook

- Make sure `xgboost_automated.ipynb` or `xgboost_automated.py` files are in good shape to be pushed to Algorithmia. Remember that if you check-in xgboost_automated.py in this repository, Github Action will take this file instead of converting `xgboost_automated.ipynb` to a `.py` file.

- Make sure Algorithmia API Key is not committed and pushed in any file!